In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import sys,hashlib
from lxml import html
from datetime import datetime

In [9]:
pd.set_option('display.max_rows', 10)
sql_max_int = 2147483647

In [4]:
response = requests.get('https://www.baseball-reference.com/teams/tgl.cgi?team=PHI&t=p&year=2020')
soup = BeautifulSoup(response.content, "lxml")
table = soup.find('table', attrs=dict(id= 'team_pitching_gamelogs'))

In [12]:
data = pd.read_html(str(table))[0]

#   Rename some of the columns
data.rename(columns={'Unnamed: 3':'HomeAway',
                    'Pitchers Used (Rest-GameScore-Dec)':'StartingPitcher',
                    '#':'PitchersUsed',
                    'Pit': 'Pitches',
                    'Str': 'Strikes'}, inplace=True)

#   Drop place holder rows 
data.drop(data[data['ERA'] == 'ERA'].index, inplace=True)

#   Fix Home/Away column values
data.replace({'HomeAway': {'@':'A'}}, inplace=True)
data['HomeAway'].fillna('H', inplace=True)

#   Split result column into multiple columns
data[['Result', 'RunsFor']] = data['Rslt'].str.split(',', expand=True)
data['RunsFor'] = data['RunsFor'].str.split('-').str[0]

#   Fix Date column for double headers and put into SQLite date format
data['Date'] = data['Date'].str.split('(').str[0] + ' 2020'
data['Date'] = data['Date'].apply(lambda x: datetime.strptime(x, '%b %d %Y').strftime('%Y-%m-%d'))

#   Pull both starting pitcher and pitcher who got the decision
all_pitchers = [row.split(',') for row in data['StartingPitcher']]
data['DecidingPitcher'] = [next(string for string in row if 'W' in string or 'L' in string).split()[0] for row in all_pitchers]
data['StartingPitcher'] = [row[0].split()[0] for row in all_pitchers]

#   Generate unique game id for each game and make it the index
game_ids = [data['Date'].iloc[i] + ((data['Opp'].iloc[i] + data['R'].iloc[i] + 'PHI' + data['RunsFor'].iloc[i]) if data['HomeAway'].iloc[i] == 'H' else ('PHI' + data['RunsFor'].iloc[i] + data['Opp'].iloc[i] + data['R'].iloc[i])) for i in range(len(data.index))]
game_ids = [int(hashlib.sha256(s.encode('utf-8')).hexdigest(), 16) % sql_max_int for s in game_ids]
data['game_id'] = game_ids
data.set_index('game_id', inplace=True)

# Add team id based on 3 letter team code
team_ids = [sum([ord(char) for char in 'PHI']) for i in range(len(data.index))]
data['team_id'] = team_ids

#   Drop unneccessary columns and reorder the remains
data.drop(columns=['Rslt', 'Rk', 'Gtm'], inplace=True)
data = data[['team_id', 'Date', 'HomeAway', 'Opp', 'Result', 'RunsFor', 'H', 'R', 'ER', 'UER', 'BB', 'SO', 'HR', 'HBP', 'BF', 'Pitches', 'Strikes', 'IR',
           'IS', 'SB', 'CS', 'AB', '2B', '3B', 'IBB', 'SH', 'SF', 'ROE', 'GDP', 'PitchersUsed', 'IP', 'ERA', 'Umpire', 'StartingPitcher', 'DecidingPitcher']]
            
#   Convert numeric columns to numeric types
data.loc[:, 'RunsFor':'PitchersUsed'] = data.loc[:, 'RunsFor':'PitchersUsed'].apply(pd.to_numeric)
data.loc[:, 'IP':'ERA'] = data.loc[:, 'IP':'ERA'].apply(pd.to_numeric)

data

,team_id,Date,HomeAway,Opp,Result,RunsFor,H,R,ER,UER,BB,SO,HR,HBP,BF,Pitches,Strikes,IR,IS,SB,CS,AB,2B,3B,IBB,SH,SF,ROE,GDP,PitchersUsed,IP,ERA,Umpire,StartingPitcher,DecidingPitcher
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
880733091,225,2020-07-24,H,MIA,L,2,9,5,5,0,3,9,1,0,38,160,108,4.0,1.0,1,1,34,2,0,0,0,1,0,0,5,9.0,5.00,Tim Timmons,A.Nola,A.Nola
1657251570,225,2020-07-25,H,MIA,W,7,6,1,1,0,3,7,0,0,32,121,80,0.0,0.0,0,0,29,1,0,0,0,0,0,4,3,9.0,3.00,D.J. Reyburn,Z.Wheeler,Z.Wheeler
1030741196,225,2020-07-26,H,MIA,L,6,12,11,11,0,4,8,4,2,44,147,94,2.0,2.0,0,0,38,1,2,0,0,0,1,0,5,9.0,5.67,Ryan Blakney,V.Velasquez,C.Irvin
463611999,225,2020-08-03,A,NYY,L,3,8,6,6,0,2,8,3,1,33,137,86,0.0,0.0,1,0,30,2,0,0,0,0,0,2,5,8.0,5.91,Nic Lentz,J.Arrieta,J.Arrieta
1951238051,225,2020-08-05,H,NYY,W,11,12,7,6,1,2,3,2,0,33,125,81,2.0,0.0,0,0,31,0,0,0,0,0,0,2,4,7.0,6.21,Angel Hernandez,Z.Wheeler,Z.Wheeler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669701541,225,2020-09-22,A,WSN,L,7,9,8,7,1,3,8,2,1,35,131,79,3.0,3.0,2,0,31,3,1,0,0,0,0,0,7,7.1,5.16,Vic Carapazza,D.Hale,B.Workman
508563154,225,2020-09-23,A,WSN,W,12,6,3,3,0,1,9,1,0,33,106,75,0.0,0.0,0,0,32,2,0,0,0,0,0,0,2,9.0,5.12,Joe West,Z.Eflin,Z.Eflin
1400042113,225,2020-09-25,A,TBR,L,4,9,6,6,0,4,12,1,1,38,162,102,5.0,3.0,1,0,33,1,0,0,0,0,0,0,5,8.0,5.15,Chris Conroy,V.Velasquez,A.Morgan


In [ ]:
tree = html.fromstring(response.content)
#tree.xpath('//div/div/div/div[contains(@data-template, \'Partials/Teams/Summary\')]/p[strong[contains(text(), \'Record\')]]/a/text()')[0]
tree.xpath('//div/div/div/div[contains(@data-template, \'Partials/Teams/Summary\')]/h1/span/text()')
tree.xpath('//div/div/div/div[contains(@data-template, \'Partials/Teams/Summary\')]/p/text()[contains(.,\'-\')]')[0].split()[0].split('-')

In [ ]:
def insert_batting(conn, data, game_id):
    batting_query = []
    for team in ['home', 'away']:
        batting_data = data[team]['teamStats']['batting']
        singles = batting_data['hits'] - (batting_data['doubles'] + batting_data['triples'] + batting_data['homeRuns'])
        for stat in ['avg', 'obp', 'slg', 'ops', 'stolenBasePercentage']:
            batting_data[stat] = float(batting_data[stat]) if batting_data[stat] != '.---' else None
        batting_data = list(batting_data.values())[:-1]
        batting_query.append((game_id, data[team]['team']['id'], *batting_data, singles))
    # Insert into Batting table for game
    batting_box_score = '''
    INSERT INTO BattingGame (
        game_id,
        team_id,
        fly_outs,
        ground_outs,
        runs,
        doubles,
        triples,
        home_runs,
        strike_outs,
        walks,
        intentional_walks,
        hits,
        hit_by_pitch,
        BA,
        AB,
        OBP,
        SLG,
        OPS,
        caught_stealing,
        bases_stolen,
        stolen_base_percentage,
        ground_into_double_play,
        ground_into_triple_play,
        plate_appearances,
        total_bases,
        RBI,
        LOB,
        sac_bunts,
        sac_flies,
        catchers_interference,
        pickoffs,
        singles)
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''
    try:
        conn.cursor().executemany(batting_box_score, batting_query)
        conn.commit()
    except sqlite3.IntegrityError:
        print('Game already added')
    except Exception as e:
        db_error_cleanup(conn, e)


def insert_pitching(conn, data, game_id):
    pitching_query = []
    for team in ['home', 'away']:
        pitching_data = data[team]['teamStats']['pitching']
        singles = pitching_data['hits'] - (pitching_data['doubles'] + pitching_data['triples'] + pitching_data['homeRuns'])
        for stat in ['obp', 'era', 'inningsPitched', 'stolenBasePercentage', 'whip']:
            pitching_data[stat] = float(pitching_data[stat]) if pitching_data[stat] != '.---' else None
        for stat in ['hitBatsmen', 'groundOutsToAirouts', 'runsScoredPer9', 'homeRunsPer9']:
            pitching_data.pop(stat)
        pitching_data = list(pitching_data.values())
        pitching_query.append((game_id, data[team]['team']['id'], *pitching_data, singles))
    # Insert into Pitching table for game
    pitching_box_score = '''
    INSERT INTO PitchingGame (
        game_id,
        team_id,
        ground_outs,
        air_outs,
        runs,
        doubles,
        triples,
        home_runs,
        strike_outs,
        walks,
        intentional_walks,
        hits,
        hit_by_pitch,
        AB,
        OBP,
        caught_stealing,
        stolen_bases,
        stolen_base_percentage,
        ERA,
        IP,
        save_oppurtunities,
        earned_runs,
        WHIP,
        batter_faced,
        outs,
        complete_games,
        shutouts,
        balks,
        wild_pitches,
        pickoffs,
        RBI,
        inherited_runners,
        inherited_runners_scored,
        catchers_interference,
        sac_bunts,
        sac_flies,
        singles)
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''
    try:
        conn.cursor().executemany(pitching_box_score, pitching_query)
        conn.commit()
    except sqlite3.IntegrityError:
        print('Game already added')
    except Exception as e:
        db_error_cleanup(conn, e)